In [1]:
!pip install numpy pandas tensorflow scikit-learn matplotlib

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.svm import SVR
from sklearn.multioutput import MultiOutputRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import regularizers, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [4]:
dataset = pd.read_csv('Exp_Mn_Mw_Value.txt', sep='\t')

# 4. Separate data values
X = dataset.iloc[:, 1:5].values  # columns 1-4: Factor A-D
y = dataset.iloc[:, 5:7].values  # columns 5-6: Responses Mn, Mw


In [5]:
print("Dataset shape:", dataset.shape)
print("Features shape:", X.shape)
print("Targets shape:", y.shape)
print("\nFirst 5 rows:")
print(dataset.head())

Dataset shape: (25, 7)
Features shape: (25, 4)
Targets shape: (25, 2)

First 5 rows:
   Run  Factor A  Factor B  Factor C  Factor D  Response 1 (Experimental)  \
0    1       110         7        50        10                    1127.19   
1    2        85        13        50        10                    1024.97   
2    3       101         1       500        60                    1950.00   
3    4       101         1       500        60                    2223.17   
4    5        50        10        50        10                    1845.60   

   Response 2 (Experimental)  
0                    1321.65  
1                    1339.35  
2                    2878.90  
3                    2989.00  
4                    2690.50  


In [7]:
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.1, 
                                                  shuffle=True, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.1111, 
                                                  shuffle=True, random_state=42)
print(f"\nData split:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Validation set: {X_val.shape[0]} samples") 
print(f"Test set: {X_test.shape[0]} samples")


Data split:
Training set: 19 samples
Validation set: 3 samples
Test set: 3 samples


In [8]:
def zscore_normalize(X_train, X_val, X_test):
    """Apply z-score normalization like MATLAB's zscore function"""
    # Calculate mean and std from training data
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0, ddof=1)  # ddof=1 for sample std like MATLAB
    
    # Apply normalization
    X_train_norm = (X_train - mean) / std
    X_val_norm = (X_val - mean) / std
    X_test_norm = (X_test - mean) / std
    
    return X_train_norm, X_val_norm, X_test_norm, mean, std

# Apply z-score normalization
X_train_norm, X_val_norm, X_test_norm, X_mean, X_std = zscore_normalize(X_train, X_val, X_test)

print(f"\nAfter z-score normalization:")
print(f"Training data mean: {np.mean(X_train_norm, axis=0)}")
print(f"Training data std: {np.std(X_train_norm, axis=0, ddof=1)}")


After z-score normalization:
Training data mean: [ 0.00000000e+00 -8.18059071e-17 -4.67462326e-17 -1.63611814e-16]
Training data std: [1. 1. 1. 1.]


In [9]:
transfer_functions = {
    'tanh': 'Tansig',
    'sigmoid': 'Logsig', 
    'linear': 'Purelin',
    'relu': 'Radbas'
}

In [12]:
neuron_counts = [8, 10, 13, 18, 20, 25]
nn_results = []
# Loop over activation functions and neuron counts
for activation, name in transfer_functions.items():
    for neurons in neuron_counts:
        print(f"\nTesting {name} ({activation}) with {neurons} neurons...")
        
        # Build model with 1 hidden layer and 2 outputs
        model = Sequential()
        model.add(Input(shape=(X_train_norm.shape[1],)))
        model.add(Dense(neurons, activation=activation,  # 1 hidden layer with variable neurons
                       kernel_regularizer=regularizers.l2(0.001)))
        model.add(Dense(2, activation='linear'))  # 2 outputs: Mn, Mw
        
        # Compile model
        model.compile(optimizer=Adam(), loss='mse', metrics=['mae'])
        
        # Early stopping
        early_stop = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
        
        # Train model
        history = model.fit(X_train_norm, y_train,
                           batch_size=4,
                           epochs=100,
                           validation_data=(X_val_norm, y_val),
                           callbacks=[early_stop],
                           verbose=0,
                           shuffle=True)
        
        # Predict on test set
        y_pred = model.predict(X_test_norm, verbose=0)
        
        # Calculate metrics
        mse_overall = mean_squared_error(y_test, y_pred)
        mae_overall = mean_absolute_error(y_test, y_pred)
        r2_overall = r2_score(y_test, y_pred)
        
        # Calculate metrics for individual outputs
        mse_mn = mean_squared_error(y_test[:, 0], y_pred[:, 0])
        mse_mw = mean_squared_error(y_test[:, 1], y_pred[:, 1])
        mae_mn = mean_absolute_error(y_test[:, 0], y_pred[:, 0])
        mae_mw = mean_absolute_error(y_test[:, 1], y_pred[:, 1])
        r2_mn = r2_score(y_test[:, 0], y_pred[:, 0])
        r2_mw = r2_score(y_test[:, 1], y_pred[:, 1])
        
        # Store results
        result = {
            'activation': name,
            'neurons': neurons,
            'model_name': f"{name}_{neurons}",
            'mse_overall': mse_overall,
            'mae_overall': mae_overall,
            'r2_overall': r2_overall,
            'mse_mn': mse_mn,
            'mse_mw': mse_mw,
            'mae_mn': mae_mn,
            'mae_mw': mae_mw,
            'r2_mn': r2_mn,
            'r2_mw': r2_mw
        }
        nn_results.append(result)
        
        print(f"Overall - MSE: {mse_overall:.4f}, MAE: {mae_overall:.4f}, R²: {r2_overall:.4f}")
        print(f"Mn - MSE: {mse_mn:.4f}, MAE: {mae_mn:.4f}, R²: {r2_mn:.4f}")
        print(f"Mw - MSE: {mse_mw:.4f}, MAE: {mae_mw:.4f}, R²: {r2_mw:.4f}")



Testing Tansig (tanh) with 8 neurons...
Overall - MSE: 10712823.6106, MAE: 2869.7064, R²: -4.1357
Mn - MSE: 6885074.6007, MAE: 2392.7471, R²: -4.9223
Mw - MSE: 14540572.6205, MAE: 3346.6658, R²: -3.3492

Testing Tansig (tanh) with 10 neurons...
Overall - MSE: 10703856.9311, MAE: 2868.9193, R²: -4.1325
Mn - MSE: 6883159.7801, MAE: 2392.7829, R²: -4.9206
Mw - MSE: 14524554.0821, MAE: 3345.0557, R²: -3.3444

Testing Tansig (tanh) with 13 neurons...
Overall - MSE: 10697722.5527, MAE: 2867.6162, R²: -4.1300
Mn - MSE: 6880669.6058, MAE: 2391.9142, R²: -4.9185
Mw - MSE: 14514775.4996, MAE: 3343.3182, R²: -3.3415

Testing Tansig (tanh) with 18 neurons...
Overall - MSE: 10687471.0033, MAE: 2866.2354, R²: -4.1238
Mn - MSE: 6869560.1780, MAE: 2390.0721, R²: -4.9089
Mw - MSE: 14505381.8286, MAE: 3342.3987, R²: -3.3387

Testing Tansig (tanh) with 20 neurons...
Overall - MSE: 10685418.6615, MAE: 2865.9462, R²: -4.1244
Mn - MSE: 6873825.0844, MAE: 2390.6390, R²: -4.9126
Mw - MSE: 14497012.2385, MAE:

In [14]:
#svm
svm_kernels = ['linear', 'poly', 'rbf', 'sigmoid']
svm_results = []

for kernel in svm_kernels:
    print(f"\nTesting SVM with {kernel} kernel...")
    
    # Create SVM model with MultiOutputRegressor for 2 outputs
    if kernel == 'poly':
        svm_model = MultiOutputRegressor(SVR(kernel=kernel, degree=3, C=1.0, epsilon=0.01))
    else:
        svm_model = MultiOutputRegressor(SVR(kernel=kernel, C=1.0, epsilon=0.01))
    
    # Train SVM
    svm_model.fit(X_train_norm, y_train)
    
    # Predict
    y_pred_svm = svm_model.predict(X_test_norm)
    
    # Calculate metrics
    mse_overall = mean_squared_error(y_test, y_pred_svm)
    mae_overall = mean_absolute_error(y_test, y_pred_svm)
    r2_overall = r2_score(y_test, y_pred_svm)
    
    # Calculate metrics for individual outputs
    mse_mn = mean_squared_error(y_test[:, 0], y_pred_svm[:, 0])
    mse_mw = mean_squared_error(y_test[:, 1], y_pred_svm[:, 1])
    mae_mn = mean_absolute_error(y_test[:, 0], y_pred_svm[:, 0])
    mae_mw = mean_absolute_error(y_test[:, 1], y_pred_svm[:, 1])
    r2_mn = r2_score(y_test[:, 0], y_pred_svm[:, 0])
    r2_mw = r2_score(y_test[:, 1], y_pred_svm[:, 1])
    
    # Store results
    result = {
        'kernel': kernel,
        'mse_overall': mse_overall,
        'mae_overall': mae_overall,
        'r2_overall': r2_overall,
        'mse_mn': mse_mn,
        'mse_mw': mse_mw,
        'mae_mn': mae_mn,
        'mae_mw': mae_mw,
        'r2_mn': r2_mn,
        'r2_mw': r2_mw
    }
    svm_results.append(result)
    
    print(f"Overall - MSE: {mse_overall:.4f}, MAE: {mae_overall:.4f}, R²: {r2_overall:.4f}")
    print(f"Mn - MSE: {mse_mn:.4f}, MAE: {mae_mn:.4f}, R²: {r2_mn:.4f}")
    print(f"Mw - MSE: {mse_mw:.4f}, MAE: {mae_mw:.4f}, R²: {r2_mw:.4f}")



Testing SVM with linear kernel...
Overall - MSE: 2307954.4588, MAE: 1172.9601, R²: -0.0148
Mn - MSE: 1156711.2606, MAE: 871.9455, R²: 0.0050
Mw - MSE: 3459197.6571, MAE: 1473.9748, R²: -0.0347

Testing SVM with poly kernel...
Overall - MSE: 2323536.1146, MAE: 1176.9664, R²: -0.0230
Mn - MSE: 1169231.0188, MAE: 877.3488, R²: -0.0057
Mw - MSE: 3477841.2104, MAE: 1476.5841, R²: -0.0402

Testing SVM with rbf kernel...
Overall - MSE: 2324219.8186, MAE: 1176.9915, R²: -0.0230
Mn - MSE: 1168518.8066, MAE: 877.0440, R²: -0.0051
Mw - MSE: 3479920.8305, MAE: 1476.9391, R²: -0.0409

Testing SVM with sigmoid kernel...
Overall - MSE: 2322986.5005, MAE: 1177.0291, R²: -0.0227
Mn - MSE: 1168719.8493, MAE: 877.1282, R²: -0.0053
Mw - MSE: 3477253.1517, MAE: 1476.9301, R²: -0.0401


In [15]:
# Create comparison dataframes
nn_df = pd.DataFrame(nn_results)
svm_df = pd.DataFrame(svm_results)

print("\nNeural Network Results:")
print(nn_df[['activation', 'neurons', 'mse_overall', 'mae_overall', 'r2_overall']].round(4))

print("\nSupport Vector Machine Results:")
print(svm_df[['kernel', 'mse_overall', 'mae_overall', 'r2_overall']].round(4))



Neural Network Results:
   activation  neurons   mse_overall  mae_overall  r2_overall
0      Tansig        8  1.071282e+07    2869.7064     -4.1357
1      Tansig       10  1.070386e+07    2868.9193     -4.1325
2      Tansig       13  1.069772e+07    2867.6162     -4.1300
3      Tansig       18  1.068747e+07    2866.2354     -4.1238
4      Tansig       20  1.068542e+07    2865.9462     -4.1244
5      Tansig       25  1.067245e+07    2863.7194     -4.1161
6      Logsig        8  1.071258e+07    2869.4936     -4.1378
7      Logsig       10  1.071260e+07    2869.6855     -4.1376
8      Logsig       13  1.069886e+07    2867.2982     -4.1303
9      Logsig       18  1.069792e+07    2866.8773     -4.1277
10     Logsig       20  1.068830e+07    2865.6939     -4.1264
11     Logsig       25  1.068122e+07    2864.2954     -4.1196
12    Purelin        8  1.070522e+07    2868.5851     -4.1319
13    Purelin       10  1.069708e+07    2867.6648     -4.1298
14    Purelin       13  1.068478e+07    2865.